In [ ]:
import pandas as pd
import numpy as np

In [2]:
df_synthetic = pd.read_csv('/content/d_synthetic.csv')

In [3]:
df_synthetic.head(1)

,RENDERING_NPI,POSTAL_CODE,DELIVERY_SYSTEM_ENCODED,PROVIDER_TYPE_ENCODED,AGE_GROUP_ENCODED,PREV,TXMT,ADV,CASE_ID,REQUIRED_SPEC,EXPEDITED,REQUIRED_LAB_TYPE,LAB_ID,LAB_POSTAL_CODE
0,1952483117,94538,0,0,0,1,0,0,1379044,1,0,0,488,94461


In [4]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 10.1 MB/s eta 0:00:00


In [5]:
lab_counts = df_synthetic['LAB_ID'].value_counts()

lower_q = 0.25
upper_q = 0.75

MIN_COUNT_THRESHOLD = lab_counts.quantile(lower_q)
MAX_COUNT_THRESHOLD = lab_counts.quantile(upper_q)

valid_lab_ids = lab_counts[
    (lab_counts >= MIN_COUNT_THRESHOLD) & (lab_counts <= MAX_COUNT_THRESHOLD)
].index.tolist()

df_filtered = df_synthetic[
    df_synthetic['LAB_ID'].isin(valid_lab_ids)
].copy()

In [6]:
df_filtered['LAB_ID'].value_counts().describe()

,count
count,119.000000
mean,4951.134454
std,1781.051089
min,2520.000000
25%,3510.500000
50%,4515.000000
75%,6411.000000
max,8631.000000


getting new counts for CTGAN process

In [7]:
lab_counts = df_filtered['LAB_ID'].value_counts()

lower_q = 0.25

ct_min = lab_counts.quantile(lower_q)

ct_lab_ids = lab_counts[(lab_counts <= ct_min)].index.tolist()

ct_filtered = df_synthetic[
    df_synthetic['LAB_ID'].isin(ct_lab_ids)
].copy()

In [8]:
ct_filtered.head(1)

,RENDERING_NPI,POSTAL_CODE,DELIVERY_SYSTEM_ENCODED,PROVIDER_TYPE_ENCODED,AGE_GROUP_ENCODED,PREV,TXMT,ADV,CASE_ID,REQUIRED_SPEC,EXPEDITED,REQUIRED_LAB_TYPE,LAB_ID,LAB_POSTAL_CODE
24,1356762934,93550,0,0,0,0,1,0,516635,0,0,0,3007,93642


In [9]:
target_samples = ct_min - min(df_filtered['LAB_ID'].value_counts())
target_samples

np.float64(990.5)

In [13]:
123

123

In [11]:
import pandas as pd
import numpy as np
from sdv.single_table import CTGANSynthesizer
from sdv.metadata import Metadata

CTGAN_EPOCHS = 300

def augment_data_with_ctgan(df_input, target_samples):

    # --- 1. Define Corrected Metadata ---
    metadata_val = {
        # CRITICAL: Define the primary key as 'id'
        'primary_key': 'CASE_ID',
        'columns': {

            'CASE_ID': {'sdtype': 'id'},
            'RENDERING_NPI': {'sdtype': 'categorical'},
            'POSTAL_CODE': {'sdtype': 'categorical'},
            'LAB_POSTAL_CODE': {'sdtype': 'categorical'},


            'LAB_ID': {'sdtype': 'categorical'},
            'REQUIRED_SPEC': {'sdtype': 'categorical'},
            'REQUIRED_LAB_TYPE': {'sdtype': 'categorical'},
            'DELIVERY_SYSTEM_ENCODED': {'sdtype': 'categorical'},
            'PROVIDER_TYPE_ENCODED': {'sdtype': 'categorical'},
            'AGE_GROUP_ENCODED': {'sdtype': 'categorical'},

            'PREV': {'sdtype': 'categorical'},
            'TXMT': {'sdtype': 'categorical'},
            'ADV': {'sdtype': 'categorical'},
            'EXPEDITED': {'sdtype': 'categorical'},
        }
    }

    for col in metadata_val['columns']:
        if col in df_input.columns:
            df_input[col] = df_input[col].astype(str)

    # 3. Load Metadata Object (Corrected path)
    metadata_object = Metadata.load_from_dict(metadata_val)

    # 4. Initialize and Train the CT-GAN Model
    synthesizer = CTGANSynthesizer(
        metadata=metadata_object,
        epochs=CTGAN_EPOCHS,
        verbose=True
    )

    synthesizer.fit(df_input)
    synthetic_data = synthesizer.sample(num_rows=target_samples)
    synthetic_data['LAB_ID'] = pd.to_numeric(synthetic_data['LAB_ID'], errors='coerce').astype(int)

    return synthetic_data

df_synthetic_augmented = augment_data_with_ctgan(ct_filtered, target_samples)

/usr/local/lib/python3.12/dist-packages/sdv/metadata/metadata.py:205: UserWarning: No table name was provided to metadata containing only one table. Assigning name: table
  warnings.warn(


PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name    Est # of Columns (CTGAN)
RENDERING_NPI           3616
POSTAL_CODE             868
DELIVERY_SYSTEM_ENCODED 3
PROVIDER_TYPE_ENCODED   2
AGE_GROUP_ENCODED       2
PREV                    2
TXMT                    2
ADV                     2
REQUIRED_SPEC           3
EXPEDITED               2
REQUIRED_LAB_TYPE       3
LAB_ID                  30
LAB_POSTAL_CODE         30

We recommend preprocessing discrete columns that can have many values, using 'update_transformers'. Or you may drop columns that are not necessary to model. (Exit this script using ctrl-C)


Gen. (0.02) | Discrim. (0.01): 100%|██████████| 300/300 [2:20:10<00:00, 28.04s/it]


TypeError: Error: Sampling terminated. No results were saved due to unspecified "output_file_path".